# tripadvisor 식당 댓글 수집

## 최근의 웹 트렌드

- HTML + 프로그램(JavaScript)
- 웹 페이지의 내용이 동적으로 변화
- 단순히 HTML만 긁어서 내용 파악이 불가능

## 트립어드바이저

https://www.tripadvisor.co.kr/Restaurant_Review-g294197-d1371740-Reviews-Mugyodong_Bugeokukjib-Seoul.html

- 리뷰의 모든 내용은 '더보기' 버튼을 클릭해야 볼 수 있음
- 개발자 도구에서 Network를 확인해보면 더보기 버튼 클릭시 통신이 이뤄지며 리뷰 내용을 가져오는 것을 확인

### 크롬드라이버 실행 및 연결

In [63]:
from selenium.webdriver import Chrome
browser = Chrome()
url = 'https://www.tripadvisor.co.kr/Restaurant_Review-g294197-d1371740-Reviews-Mugyodong_Bugeokukjib-Seoul.html'
browser.get(url)

### 더보기 링크 찾기

In [64]:
more_links = browser.find_elements_by_css_selector('.ulBlueLinks')

In [65]:
# 더보기 링크 클릭
    #하나만 클릭해도 전체가 실행된다. 만약 각각 클릭해야 하는 경우는 아래와 같이 반복문을 사용
more_links[0].click()
# for link in more_links:
#     try:
#         link.click()
#     except Exception as e:
#         print(e)

## 리뷰 스크래핑

- `.text`로 텍스트 확인

In [66]:
reviews = browser.find_elements_by_css_selector('.partial_entry') # 리뷰 써있는 부분

In [69]:
data = []
for review in reviews:
    data.append(review.text)
    #print(review.text)
    
import pandas as pd
pd.DataFrame({'text':data}).to_csv("리뷰.csv", encoding='euc-kr')

## TODO 
- 페이지 클릭해서 모든 댓글 가져오기
![페이징](img/tripadvisor_paging.png)

- 위처럼 23이 마지막 페이지고 **다음** 버튼 클릭하면 다음으로 넘어간다. 

- 계획
    1. 마지막 페이지 번호를 읽어온다.
    2. 1 ~ 마지막페이지 까지 반복문을 돌린다.
    3. 반복문에서 댓글읽고 다음 버튼 클릭하고를 반복한다.

- review 더보기 : .ulBlueLinks
- review들 : .partial_entry
- 마지막 페이지번호 
    - 페이징블럭 : div.mobile-more div.pageNumbers > a:last-child 
- 다음버튼 찾기 : div.mobile-more  a.nav.next.taLnk.ui_button.primary  (a.nav.next.taLnk.ui_button.primary인데 이게 아래 하나 더 있어서 조상을 찾음)


In [54]:
from selenium.webdriver import Chrome, ChromeOptions
import time

In [55]:
url = 'https://www.tripadvisor.co.kr/Restaurant_Review-g294197-d1371740-Reviews-Mugyodong_Bugeokukjib-Seoul.html'

# Headless

In [70]:
options = ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')

In [71]:
browser = Chrome(options=options)
browser.get(url)

In [72]:
# 마지막 페이지 번호, 다음버튼 찾기
last_page = browser.find_element_by_css_selector('div.mobile-more div.pageNumbers > a:last-child ').text.strip()
print(last_page)

23


# Guide page 로딩 시간을 충분히 주지 않으면 예외발생.
- 수업엔 0.5하니 괜찮네
- 페이지가 로딩되기 전에 클릭되어서
- https://stackoverflow.com/questions/12967541/how-to-avoid-staleelementreferenceexception-in-selenium/12967602 여기 반복문 사용하는 구문 참조

In [73]:
review_list = []
cnt = int(last_page) #문자열 -> 정수

while cnt > 0:
    attempts = 0
    try:
        more_btns = browser.find_elements_by_css_selector('.ulBlueLinks')
        #print(more_btns[0].text)
        more_btns[0].click()
        time.sleep(0.7) #적당히 기다리지 않으면 에러난다. 
        reviews = browser.find_elements_by_css_selector('.partial_entry')
        for review in reviews:
            review_list.append(review.text)

       
        # 페이지가 바뀔때 마다 다시 조회해야 한다.(#StaleElementReferenceException 발생)
        next_btn = browser.find_element_by_css_selector('div.mobile-more  a.nav.next.taLnk.ui_button.primary')
        next_btn.click()
        time.sleep(0.5)
    except Exception as e:
        attempts += 1
        
    cnt -= 1    

더 보기
더 보기
숨기기
숨기기
숨기기
더 보기
더 보기
더 보기
더 보기
더 보기
더 보기
더 보기
더 보기
숨기기
숨기기
숨기기
숨기기
숨기기
숨기기
숨기기
더 보기
숨기기
숨기기


In [61]:
len(review_list)

158

In [62]:
browser.close()